## Clean listings dataset

### Framwork:
0. Import listings dataset
1. Prepare training and holdout datasets
2. Explore raw listings dataset
3. Clean raw listings dataset
4. Wrap data cleaning using functions 

## 0. Import listings dataset

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import json
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

from tqdm.notebook import tqdm
from langdetect import detect

pd.set_option('display.max_columns', 100)

RANDOM_STATE= 42

In [2]:
listing_df_raw = pd.read_csv('../data/data_raw/listings.csv.gz', compression='gzip')

## 1. Prepare training and holdout datasets 

To avoid **data leakage**, we separate the listings dataset first then do data cleaning to avoid filling nan values with calculated imputation. We will do the same cleaning for the holdout dataset when we use it.

Randomly select 80% of the raw listings dataset as the training dataset, and the rest as the holdout dataset.

In [3]:
# split data into training and holdout datasets with a ratio of 8:2
listing_train = listing_df_raw.sample(frac=0.8, random_state = RANDOM_STATE)
listing_holdout = listing_df_raw.loc[~listing_df_raw.index.isin(listing_train.index)]
listing_train.reset_index(drop=True, inplace=True)
listing_holdout.reset_index(drop=True, inplace=True)
print(listing_train.shape, listing_holdout.shape)

(4933, 75) (1233, 75)


In [4]:
# save the training and holdout listing datasets
listing_train.to_pickle('../data/data_cleaned/listing_train.zip')
listing_holdout.to_pickle('../data/data_cleaned/listing_holdout.zip')

## 2. Explore raw listings dataset

First, let's explore the raw Seattle Airbnb listings dataset downloaded from Inside Airbnb.

In [5]:
listing_train.shape

(4933, 75)

In [6]:
listing_train.head(2)

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bathrooms_text,bedrooms,beds,amenities,price,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,49113826,https://www.airbnb.com/rooms/49113826,20221224045325,2022-12-24,city scrape,Private Seattle Getaway Home Near Lake Washing...,A Seattle vacation destination for those from ...,SeaTac International Airport: 14 min drive<br ...,https://a0.muscache.com/pictures/miso/Hosting-...,188538325,https://www.airbnb.com/users/show/188538325,Xiao,2018-05-08,"Seattle, WA",NaN,within an hour,100%,100%,t,https://a0.muscache.com/im/pictures/user/32fd3...,https://a0.muscache.com/im/pictures/user/32fd3...,Oakland,4,4,"['email', 'phone']",t,t,"Seattle, Washington, United States",Rainier View,Rainier Valley,47.510280,-122.246680,Entire home,Entire home/apt,8,NaN,2 baths,4.0,4.0,"[""Hot water"", ""Dining table"", ""Toaster"", ""Refr...",$280.00,3,30,3.0,3.0,1125.0,1125.0,3.0,1125.0,NaN,t,23,53,83,341,2022-12-24,18,7,1,2021-06-22,2022-11-29,4.83,4.89,4.94,4.89,4.89,4.61,4.72,STR-OPLI-21-000235,f,1,1,0,0,0.98
1,7455832,https://www.airbnb.com/rooms/7455832,20221224045325,2022-12-24,city scrape,Classic remodeled in Georgetown,Enjoy your Seattle stay in the vibrant & artis...,Georgetown is a cool neighborhood about three ...,https://a0.muscache.com/pictures/miso/Hosting-...,2144954,https://www.airbnb.com/users/show/2144954,Ryan,2012-04-15,"Seattle, WA",NaN,within an hour,100%,100%,f,https://a0.muscache.com/im/pictures/user/a6711...,https://a0.muscache.com/im/pictures/user/a6711...,Greater Duwamish,2,2,"['email', 'phone', 'work_email']",t,f,"Seattle, Washington, United States",Georgetown,Other neighborhoods,47.544739,-122.319786,Entire home,Entire home/apt,6,NaN,2 baths,3.0,3.0,"[""Hot water"", ""Crib - available upon request"",...",$156.00,1,150,2.0,3.0,1125.0,1125.0,2.1,1125.0,NaN,t,17,35,65,329,2022-12-24,27,27,3,2022-06-09,2022-12-13,4.89,4.96,4.89,4.93,4.96,4.93,4.93,STR-OPLI-22-000359,t,2,2,0,0,4.07


In [7]:
listing_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4933 entries, 0 to 4932
Data columns (total 75 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            4933 non-null   int64  
 1   listing_url                                   4933 non-null   object 
 2   scrape_id                                     4933 non-null   int64  
 3   last_scraped                                  4933 non-null   object 
 4   source                                        4933 non-null   object 
 5   name                                          4933 non-null   object 
 6   description                                   4925 non-null   object 
 7   neighborhood_overview                         3591 non-null   object 
 8   picture_url                                   4933 non-null   object 
 9   host_id                                       4933 non-null   i

In [8]:
listing_train.last_scraped.unique()

array(['2022-12-24'], dtype=object)

In [9]:
listing_train.describe()

,id,scrape_id,host_id,host_listings_count,host_total_listings_count,latitude,longitude,accommodates,bathrooms,bedrooms,beds,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,availability_30,availability_60,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
count,4.933000e+03,4.933000e+03,4.933000e+03,4933.000000,4933.000000,4933.000000,4933.000000,4933.000000,0.0,4362.000000,4879.000000,4933.000000,4933.000000,4931.000000,4931.000000,4.931000e+03,4.931000e+03,4931.000000,4.931000e+03,0.0,4933.000000,4933.000000,4933.000000,4933.000000,4933.000000,4933.000000,4933.000000,4095.000000,4093.000000,4093.000000,4093.000000,4093.000000,4093.000000,4093.000000,4933.000000,4933.000000,4933.000000,4933.000000,4095.000000
mean,2.184880e+17,2.022122e+13,1.284527e+08,398.312183,463.601054,47.626241,-122.334295,3.977498,NaN,1.779917,2.126460,12.531928,525.340564,12.777530,35.556682,8.718106e+05,8.718365e+05,34.082803,8.718249e+05,NaN,15.466856,35.858099,57.110683,200.418204,64.054936,17.699372,0.850395,4.787245,4.832277,4.807826,4.871503,4.862451,4.847603,4.718067,38.526454,37.018853,1.498682,0.008514,2.282764
std,3.178194e+17,0.000000e+00,1.458478e+08,1200.944835,1349.721244,0.049561,0.033030,2.511632,NaN,1.064824,1.460072,18.337902,500.767790,24.533361,89.764724,4.324473e+07,4.324473e+07,86.897957,4.324473e+07,NaN,11.112725,21.546846,31.282007,128.485828,102.209764,23.239973,1.570046,0.353675,0.314753,0.327187,0.309831,0.333105,0.249373,0.355027,102.203151,102.372463,5.188797,0.149100,1.988425
min,6.606000e+03,2.022122e+13,2.536000e+03,1.000000,1.000000,47.497920,-122.419050,0.000000,NaN,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000e+00,1.000000e+00,1.000000,1.000000e+00,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.010000
25%,2.400051e+07,2.022122e+13,1.538198e+07,1.000000,1.000000,47.601790,-122.356770,2.000000,NaN,1.000000,1.000000,2.000000,31.000000,2.000000,2.000000,3.650000e+02,3.650000e+02,2.000000,3.650000e+02,NaN,4.000000,18.000000,34.000000,81.000000,2.000000,1.000000,0.000000,4.740000,4.800000,4.760000,4.880000,4.880000,4.810000,4.660000,1.000000,1.000000,0.000000,0.000000,0.670000
50%,4.935400e+07,2.022122e+13,6.838571e+07,3.000000,4.000000,47.622580,-122.333100,4.000000,NaN,1.000000,2.000000,3.000000,365.000000,2.000000,3.000000,1.125000e+03,1.125000e+03,3.000000,1.125000e+03,NaN,17.000000,43.000000,67.000000,180.000000,22.000000,7.000000,0.000000,4.890000,4.920000,4.910000,4.960000,4.960000,4.920000,4.800000,2.000000,1.000000,0.000000,0.000000,1.810000
75%,6.137287e+17,2.022122e+13,1.905074e+08,18.000000,25.000000,47.663770,-122.311550,5.000000,NaN,2.000000,3.000000,30.000000,1125.000000,30.000000,30.000000,1.125000e+03,1.125000e+03,30.000000,1.125000e+03,NaN,26.000000,55.000000,84.000000,335.000000,81.000000,28.000000,1.000000,4.980000,4.990000,5.000000,5.000000,5.000000,4.980000,4.910000,11.000000,8.000000,0.000000,0.000000,3.340000
max,7.878140e+17,2.022122e+13,4.919696e+08,4648.000000,5072.000000,47.734010,-122.240870,16.000000,NaN,11.000000,16.000000,365.000000,10000.000000,365.000000,366.000000,2.147484e+09,2.147484e+09,365.000000,2.147484e+09,NaN,30.000000,60.000000,90.000000,365.000000,1224.000000,152.000000,36.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,410.000000,410.000000,38.000000,4.000000,15.880000


In [10]:
# nan value percentage
(listing_train.isna().sum()/listing_train.shape[0]).sort_values(ascending=False)[:25]

bathrooms                      1.000000
calendar_updated               1.000000
host_about                     0.277924
neighborhood_overview          0.272045
neighbourhood                  0.272045
license                        0.226840
review_scores_cleanliness      0.170282
review_scores_accuracy         0.170282
review_scores_checkin          0.170282
review_scores_communication    0.170282
review_scores_location         0.170282
review_scores_value            0.170282
review_scores_rating           0.169876
last_review                    0.169876
first_review                   0.169876
reviews_per_month              0.169876
host_location                  0.137644
host_response_time             0.123454
host_response_rate             0.123454
bedrooms                       0.115751
host_acceptance_rate           0.072775
host_neighbourhood             0.058585
beds                           0.010947
description                    0.001622
bathrooms_text                 0.000608


In [11]:
# # nan value count
listing_train.isna().sum().sort_values(ascending=False)[:25]

bathrooms                      4933
calendar_updated               4933
host_about                     1371
neighborhood_overview          1342
neighbourhood                  1342
license                        1119
review_scores_cleanliness       840
review_scores_accuracy          840
review_scores_checkin           840
review_scores_communication     840
review_scores_location          840
review_scores_value             840
review_scores_rating            838
last_review                     838
first_review                    838
reviews_per_month               838
host_location                   679
host_response_time              609
host_response_rate              609
bedrooms                        571
host_acceptance_rate            359
host_neighbourhood              289
beds                             54
description                       8
bathrooms_text                    3
dtype: int64

## 3. Clean raw listings dataset

### 3.1 remove useless columns

removed 7 columns:
['scrape_id','source','bathrooms','host_thumbnail_url','neighbourhood','calendar_updated','calendar_last_scraped']

To do the cleaning, first let's narrow down the listing dataset to only include the features we're interested in.

In [12]:
# listing_train.columns

In [13]:
# columns to remove

cols_to_remove = ['scrape_id', 'source',
                  'bathrooms','calendar_updated', # no values
                  'host_thumbnail_url', # same host pic as host_picture_url but in a small size
                  'neighbourhood', # will use neighbourhood_cleansed
                  'calendar_last_scraped' # 2022-12-24 same values as last_scraped
                 ]

listing_df = listing_train.drop(cols_to_remove,axis=1)
print(listing_df.shape)
print('removed {} columns'.format(len(cols_to_remove)))
# listing_df.head(3)

(4933, 68)
removed 7 columns


### 3.2 clean features
#### clean bathrooms_text column


For ['bathrooms_text'], we extract number of baths from string and convert them to numericals. After that we remove this column.

In [14]:
# example of the raw bathrooms_text col
listing_df.bathrooms_text.unique()

array(['2 baths', '1 bath', '1.5 baths', '3 baths', '3.5 baths',
       '1 private bath', '1 shared bath', '1.5 shared baths', '2.5 baths',
       '0 shared baths', '2 shared baths', '3 shared baths', nan,
       'Half-bath', '0 baths', '2.5 shared baths', '6 baths', '6.5 baths',
       '16 shared baths', '4 baths', '4.5 baths', 'Private half-bath',
       '7 baths', 'Shared half-bath', '3.5 shared baths'], dtype=object)

In [15]:
# extract the num of bathrooms from bathrooms_text

replace_dict = dict(zip(['half-bath','baths','bath'],['0.5','','']))
cleaned_bathrooms_series = listing_df['bathrooms_text'].str.lower().replace(replace_dict, regex=True).str.strip()
# extract all the int and float 
listing_df['bathrooms_count'] = cleaned_bathrooms_series.str.extract('(\d*\.?\d+)').astype('float64')
listing_df.drop(['bathrooms_text'], axis=1, inplace=True)

#### clean license column
['license']
convert license column values into True and False, which 1 represents a listing has a license and 0 represents a listing does ont have a license.

In [16]:
# convert license into True and False
listing_df['license'] = listing_df['license'].notnull().astype('int64')
listing_df.rename(columns = {'license': 'has_license'}, inplace=True)

#### clean binary-like columns 
['host_is_superhost', 'host_has_profile_pic', 'host_identity_verified', 'has_availability', 'instant_bookable']

For these columns, we convert the string t/f to 1/0.

In [17]:
# convert binary cols t/f to 1/0 
binary_cols = listing_df.columns[listing_df.isin(['t','f']).any()].to_list()
# print(binary_cols)
listing_df.loc[:,binary_cols] = listing_df.loc[:,binary_cols].fillna(0).replace(({'t':1, 'f':0}))

#### clean numeric-like columns 
['host_response_rate', 'host_acceptance_rate', 'price']
For these columns, we extract numbers from the string values and convert them into integers.

In [18]:
# extract num from text-based num cols and convert to int

listing_df['host_response_rate'] = listing_df['host_response_rate'].str.rstrip('%').astype(float)/100
listing_df['host_acceptance_rate'] = listing_df['host_acceptance_rate'].str.rstrip('%').astype(float)/100

listing_df['price'] = listing_df['price'].str.replace('$', '')
listing_df['price'] = listing_df['price'].replace(to_replace ='\.00', value = '', regex = True)
listing_df['price'] = listing_df['price'].replace(to_replace =',', value = '', regex = True)
listing_df['price'] = listing_df['price'].astype(float)

#### clean date-like columns 
['last_scraped', 'host_since','first_review','last_review']

For these columns, we convert the string date to date datatype.

In [19]:
# convert date cols to date type

date_cols = ['last_scraped', 'host_since','first_review','last_review']
listing_df.loc[:,date_cols] = listing_df.loc[:,date_cols].apply(pd.to_datetime, format='%Y-%m-%d')

#### clean id column
Convert the numeric id column to categorical column.

In [20]:
# convert id datatype to object

listing_df['id'] = listing_df['id'].astype('object')
listing_df['host_id'] = listing_df['host_id'].astype('object')

#### rename some columns

In [21]:
# rename columns
listing_df = listing_df.rename(columns = {'id': 'listing_id', 
                                          'name': 'listing_name',
                                         })

### 3.3 handle missing values

In [22]:
# nan value count
# listing_df.isna().sum().sort_values(ascending=False)[:30]

#### fill nans for host_response_time
For this column, we fill nans with **'no response'** since host_response_time is a categorical feature. 

In [23]:
#fillna for host_response_time
listing_df['host_response_time'] = listing_df['host_response_time'].fillna('no response')

#### fill nans for bathrooms
Since there only 3 nan values, **we will fill these nans with 
0,1,1** by looking up the number of bedrooms from Airbnb website using the listing ids manuially. 


In [24]:
# fillna values for the bathrooms
# get the url of the listings with nan values in bathroom
# listing_df[listing_df.bathrooms_text.isna()]['listing_url']
naval_idx = listing_df[listing_df.bathrooms_count.isna()].index
listing_df.loc[naval_idx,'bathrooms_count'] = [0,1,1]

#### fill nans for datetime columns
We fill all the nans with mode for the datetime columns

In [25]:
date_cols = list(listing_df.select_dtypes(include='datetime').columns)
# print(date_cols)
for col in date_cols:
    listing_df[col].fillna(listing_df[col].mode()[0], inplace=True)

In [26]:
# listing_df.isna().sum().sort_values(ascending=False)[:30]

#### fill nans  for review_cols 
We fill all nans with 0 for all the numeric review related columns since there are zero comments.

In [27]:
numerics = ['int64','float64']
review_cols = [col for col in listing_df.select_dtypes(include=numerics).columns if 'review' in col]
# print(review_cols)
listing_df.loc[:,review_cols] = listing_df[review_cols].fillna(0)

#### fill nans with mean for host_cols
We fill all nans with means for the host_response_rate and host_acceptance_rate columns.

In [28]:
listing_df['host_response_rate'].fillna(listing_df['host_response_rate'].mean(), inplace=True)
listing_df['host_acceptance_rate'].fillna(listing_df['host_acceptance_rate'].mean(), inplace=True)

#### fill nans with mini_max_night_cols
We fill all nans with the means for these columns.

In [29]:
mini_max_night_cols = ['minimum_minimum_nights','minimum_maximum_nights','minimum_nights_avg_ntm',
                       'maximum_maximum_nights','maximum_minimum_nights','maximum_nights_avg_ntm']

for col in mini_max_night_cols:
     listing_df[col].fillna(listing_df[col].mean(), inplace=True)

#### fill nans for bedrooms col
Fill bedrooms with the same value from beds, then fill the rest nans with 1 for bedrooms and beds.

In [30]:
listing_df['bedrooms'].fillna(listing_df['beds'], inplace=True)
listing_df.loc[:,['bedrooms','beds']] = listing_df.loc[:,['bedrooms','beds']].fillna(1)

#### fill " " for all the nans in the categorical cols

In [31]:
text_cols = listing_df.select_dtypes(include='object').columns.to_list()
listing_df[text_cols] = listing_df[text_cols].fillna(' ')

## 4. Wrap data cleaning using functions 

Here we make a couple of functions to wrap the data cleaning steps we took above for easy access to the holdout dataset later.

In [32]:
def remove_columns(df):
    cols_to_remove = ['scrape_id', 'source',
                      'bathrooms','calendar_updated', # no values
                      'host_thumbnail_url', # same host pic as host_picture_url but in a small size
                      'neighbourhood', # will use neighbourhood_cleansed
                      'calendar_last_scraped' # 2022-12-24 same values as last_scraped
                     ]
    new_df = df.drop(cols_to_remove,axis=1)
    
    return new_df


In [33]:
def clean_columns(df):
    
    # extract the num of bathrooms from bathrooms_text
    replace_dict = dict(zip(['half-bath','baths','bath'],['0.5','','']))
    cleaned_bathrooms_series = df['bathrooms_text'].str.lower().replace(replace_dict, regex=True).str.strip()
    # extract all the int and float 
    df['bathrooms_count'] = cleaned_bathrooms_series.str.extract('(\d*\.?\d+)').astype('float64')
    df.drop(['bathrooms_text'], axis=1, inplace=True)
    
    # convert license into True and False
    df['license'] = df['license'].notnull().astype('int64')
    df.rename(columns = {'license': 'has_license'}, inplace=True)
    
    # convert binary cols t/f to 1/0 
    binary_cols = df.columns[df.isin(['t','f']).any()].to_list()
    # print(binary_cols)
    df.loc[:,binary_cols] = df.loc[:,binary_cols].fillna(0).replace(({'t':1, 'f':0}))
    
    # extract num from text-based num cols and convert to int
    df['host_response_rate'] = df['host_response_rate'].str.rstrip('%').astype(float)/100
    df['host_acceptance_rate'] = df['host_acceptance_rate'].str.rstrip('%').astype(float)/100

    df['price'] = df['price'].str.replace('$', '')
    df['price'] = df['price'].replace(to_replace ='\.00', value = '', regex = True)
    df['price'] = df['price'].replace(to_replace =',', value = '', regex = True)
    df['price'] = df['price'].astype(float)
    
    # convert date cols to date type
    date_cols = ['last_scraped', 'host_since','first_review','last_review']
    df.loc[:,date_cols] = df.loc[:,date_cols].apply(pd.to_datetime, format='%Y-%m-%d')
    
    # convert id datatype to object
    df['id'] = df['id'].astype('object')
    df['host_id'] = df['host_id'].astype('object')
    
    # rename columns
    df = df.rename(columns = {'id': 'listing_id', 'name': 'listing_name'})
    
    return df

In [34]:
def handle_missing_values(df):

    #fillna for host_response_time
    # For this column, we fill nans with 'no response' 
    # since host_response_time is a categorical feature.
    df['host_response_time'] = df['host_response_time'].fillna('no response')
    
    # fillna values for the bathrooms
    # Since there only 3 nan values, we will fill these nans with 0,1,1 
    # by looking up the number of bedrooms from Airbnb website 
    # using the listing ids manuially.
    naval_idx = listing_df[listing_df.bathrooms_count.isna()].index
    listing_df.loc[naval_idx,'bathrooms_count'] = [0,1,1]
    
    # fill nans with mode for date type cols
    date_cols = list(df.select_dtypes(include='datetime').columns)
    # print(date_cols)
    for col in date_cols:
        df[col].fillna(df[col].mode()[0], inplace=True)

    # fill nans with 0 for review_cols since there are zero comments
    numerics = ['int64','float64']
    review_cols = [col for col in df.select_dtypes(include=numerics).columns if 'review' in col]
    # print(review_cols)
    df.loc[:,review_cols] = df[review_cols].fillna(0)

    # fill nans with mean for host_cols
    df['host_response_rate'].fillna(df['host_response_rate'].mean(), inplace=True)
    df['host_acceptance_rate'].fillna(df['host_acceptance_rate'].mean(), inplace=True)

    # fill nans with the same value from minimum_nights and maximum_nights cols
    mini_max_night_cols = ['minimum_minimum_nights','minimum_maximum_nights','minimum_nights_avg_ntm',
                           'maximum_maximum_nights','maximum_minimum_nights','maximum_nights_avg_ntm']
    for col in mini_max_night_cols:
         df[col].fillna(df[col].mean(), inplace=True)

    # fill nans with the same value from beds
    # then fill the rest nans with 1 for bedrooms and beds
    df['bedrooms'].fillna(df['beds'], inplace=True)
    df.loc[:,['bedrooms','beds']] = df.loc[:,['bedrooms','beds']].fillna(1)

    # fill " " for all the nans in the categorical cols
    text_cols = df.select_dtypes(include='object').columns.to_list()
    df[text_cols] = df[text_cols].fillna(' ')
    
    return df

In [35]:
def add_new_features(df):

    # new feature amenities_count
    # convert string list to list
    df['amenities_count'] = df['amenities'].apply(lambda x: len(json.loads(x)))
    
    # new feature host_response_time_encoded
    dict_map = {'within an hour': 1,
                'within a few hours': 2,
                'within a day': 3,
                'a few days or more': 4,
                'no response':5}
    df['host_response_time_encoded'] = df['host_response_time'].map(dict_map)

    # new feature host_operate_years
    delta = df['last_scraped'] - df['host_since']
    df['host_operate_years'] = (delta/365).apply(lambda x: x.days)
    
    return df

In [36]:
listing_df = remove_columns(listing_train)
listing_df = clean_columns(listing_df)
listing_df = handle_missing_values(listing_df)
listing_df = add_new_features(listing_df)

In [37]:
print(listing_df.shape)

(4933, 71)


In [38]:
listing_df.isna().sum().sort_values(ascending=False)[:10]

listing_id                0
availability_60           0
first_review              0
number_of_reviews_l30d    0
number_of_reviews_ltm     0
number_of_reviews         0
availability_365          0
availability_90           0
availability_30           0
review_scores_rating      0
dtype: int64

In [39]:
# save the cleaned listing data
listing_df.to_pickle('../data/data_cleaned/cleaned_listing.zip')

In [40]:
# check and read the saved data
cleaned_listing_df = pd.read_pickle('../data/data_cleaned/cleaned_listing.zip')
print(cleaned_listing_df.shape)
cleaned_listing_df.head(2)

(4933, 71)


,listing_id,listing_url,last_scraped,listing_name,description,neighborhood_overview,picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,property_type,room_type,accommodates,bedrooms,beds,amenities,price,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,has_availability,availability_30,availability_60,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,has_license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,bathrooms_count,amenities_count,host_response_time_encoded,host_operate_years
0,49113826,https://www.airbnb.com/rooms/49113826,2022-12-24,Private Seattle Getaway Home Near Lake Washing...,A Seattle vacation destination for those from ...,SeaTac International Airport: 14 min drive<br ...,https://a0.muscache.com/pictures/miso/Hosting-...,188538325,https://www.airbnb.com/users/show/188538325,Xiao,2018-05-08,"Seattle, WA",,within an hour,1.0,1.0,1,https://a0.muscache.com/im/pictures/user/32fd3...,Oakland,4,4,"['email', 'phone']",1,1,Rainier View,Rainier Valley,47.510280,-122.246680,Entire home,Entire home/apt,8,4.0,4.0,"[""Hot water"", ""Dining table"", ""Toaster"", ""Refr...",280.0,3,30,3.0,3.0,1125.0,1125.0,3.0,1125.0,1,23,53,83,341,18,7,1,2021-06-22,2022-11-29,4.83,4.89,4.94,4.89,4.89,4.61,4.72,1,0,1,1,0,0,0.98,2.0,53,1,4
1,7455832,https://www.airbnb.com/rooms/7455832,2022-12-24,Classic remodeled in Georgetown,Enjoy your Seattle stay in the vibrant & artis...,Georgetown is a cool neighborhood about three ...,https://a0.muscache.com/pictures/miso/Hosting-...,2144954,https://www.airbnb.com/users/show/2144954,Ryan,2012-04-15,"Seattle, WA",,within an hour,1.0,1.0,0,https://a0.muscache.com/im/pictures/user/a6711...,Greater Duwamish,2,2,"['email', 'phone', 'work_email']",1,0,Georgetown,Other neighborhoods,47.544739,-122.319786,Entire home,Entire home/apt,6,3.0,3.0,"[""Hot water"", ""Crib - available upon request"",...",156.0,1,150,2.0,3.0,1125.0,1125.0,2.1,1125.0,1,17,35,65,329,27,27,3,2022-06-09,2022-12-13,4.89,4.96,4.89,4.93,4.96,4.93,4.93,1,1,2,2,0,0,4.07,2.0,55,1,10


### Below are the columns considered to remove 

I keep the most columns in the cleaned_listing dataset. Below are the columns we consider to remove for the next step. Please check and add the columns yuo think should be dropped.

In [41]:
feature_to_remove = ['host_total_listings_count','host_listings_count', # same description with different values, use calculated_host_listings_count instead
                    'minimum_minimum_nights', 'maximum_minimum_nights', # those num are from calender
                    'minimum_maximum_nights', 'maximum_maximum_nights', # they're constantly changing 
                    'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', # so do not make much sense

                    'has_availability','availability_30',
                    'availability_60', 'availability_90','availability_365',
                    'reviewer_count', #already has it 
                    'host_number_of_year', # rename it as host_operating_years #RENAMEED
                    ]
                    






<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=f2a50dc6-ff6a-45ff-9dbe-d7a35bd1e393' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>